## Sampling documents

In [ ]:
import glob
import os

DPT_S = "[dptment_start]---------------"
DPT_E = "[dptment_end]---------------"
DOC_S = "[doc_start]---------------"
DOC_E = "[doc_end]---------------"

In [ ]:
data_category = ["visits_2011to2020"]

all_pts = []
path = "./data"
for d in range(len(data_category)):
    print("data_category[d]: ", data_category[d])
    target_folder = path +"/"+ data_category[d]
    data_folders = glob.glob(target_folder+"/*")
    data_folders.sort()
    
    for f in range(len(data_folders)):
        group = data_folders[f]
        pateints = glob.glob(group+"/*.txt")
#         print("group: ", group)
        
        for p in range(len(pateints)):
            if pateints[p] not in all_pts:
                all_pts.append(pateints[p])

## get patient_pools

In [ ]:
def get_patient_pools(path):
    file = open(path)
    lines = file.readlines()
    file.close()

    _pts = []

    for l in range(len(lines)):
        line = lines[l].strip("\n")
        _pts.append(line.split("/")[-1].split(".")[0])
    return _pts


train_pts = get_patient_pools("../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_train.txt")
test_pts = get_patient_pools("../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_test.txt")

print("len(train_pts): ", len(train_pts))
print("len(test_pts): ", len(test_pts))


## set Department
- ['내분비대사내과', '호흡기내과', '순환기내과', '소화기내과', '류마티스내과', '신장내과', '알레르기내과', '감염내과']

In [ ]:
departments = ['내분비대사내과', '호흡기내과', '순환기내과', '소화기내과', '류마티스내과', '신장내과', '알레르기내과', '감염내과']
file = open("./data/departments.txt", "w")
file.write("\n".join(departments))
file.close()

In [ ]:
departments = []

for p in range(len(all_pts)):
    file = open(all_pts[p], "r")
    lines = file.readlines()
    patient_num =all_pts[p].split("/")[-1].split(".")[0]
    
    if p%500==0:
        print(str(p) +"/"+str(len(all_pts)))
    
    dpts = []
    documents = []
    doc = []
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        if line == DPT_S:
            mode = "read_dpt"
        
        elif line == DPT_E:
            #if line not in dpts:
            dpts.append(line)
        
        elif line == DOC_S:
            mode = "read_doc"
            
        else:
            if mode == "read_dpt":
                if line not in departments:
                    departments.append(line)
        
print("departments: ", departments)

In [ ]:
print("len(departments): ", len(departments))

### read documents

In [ ]:
file = open("./data/departments.txt", "r")
lines = file.readlines()
file.close()

label_i_to_w = {}
label_w_to_i = {}
for l in range(len(lines)):
    label_i_to_w[l] = lines[l].strip("\n")
    label_w_to_i[lines[l].strip("\n")] = l 
print(label_i_to_w)
print(label_w_to_i)


## read documents

In [ ]:
docperfile = 50
def get_documents_of_dpt(ptsnums):
    department_docs = {}

    print("len(ptsnums): ", len(ptsnums))
    
    for t in range(len(ptsnums)):
#         print("ptsnums[t]: ", ptsnums[t])
        
        for n in range(len(ptsnums[t])):
            if ptsnums[t]!='0':
                ptnum_int = int(ptsnums[t][n:])
                break
#         print('ptnum_int: ', ptnum_int)
        
        target_group = str(((ptnum_int//docperfile)+1)*docperfile)
#         print("target_group: ", target_group)
        
        target_path = "./data/visits_2011to2020/"+str(target_group)+"/"+str(ptsnums[t])+".txt"
        print("target_path: ", target_path)
        
        file = open(target_path, "r")
        lines = file.readlines()
        file.close()        

        dpts = []
        documents = []
        doc = []
        department = ""
        for l in range(len(lines)):
            line = lines[l].strip("\n")
            if line == DPT_S:
                mode = "read_dpt"
            elif line == DPT_E:
                mode = ""
                
            elif line == DOC_S:
                mode = "read_doc"

            elif line == DOC_E:
                mode = ""
                if department in department_docs:
                    tmp_docs = department_docs[department]
                    tmp_docs.append(doc)
                    department_docs[department] = tmp_docs
                else:
                    department_docs[department] = [doc]

                #documents.append(doc)
                doc = []

            else:
                if mode=="read_doc":
                    doc.append(line)
                elif mode=="read_dpt":
                    department = line
                    
    return department_docs
    
    

In [ ]:
import random as rng

limit_patients = 1000
rng.shuffle(train_pts)
train_pts = train_pts[:limit_patients]

limit_patients = 100
rng.shuffle(test_pts)
test_pts = test_pts[:limit_patients]

docs_train = get_documents_of_dpt(ptsnums=train_pts)
docs_test  = get_documents_of_dpt(ptsnums=test_pts)

## 샘플 생성

In [ ]:
import random as rng

print(label_i_to_w)
print(label_w_to_i)



def sampling_data(docs_arrays, samplenum = 1000, dup=5, output_directory="./data/"):
    gen_sample_num = 0
    
    print("output_directory: ", output_directory)
    outpath = output_directory.split("/")[:-1]
    outpath = "/".join(outpath)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    
    outtext = []
    for e in range(len(label_i_to_w)):
        dpt = label_i_to_w[e]
        print("dpt: ", dpt)

        docs = docs_arrays[dpt]

        for s in range(samplenum):
            for _ in range(dup):
                target1 = rng.randint(0, len(docs)-1)
                doc1 = " ".join(docs[target1])
                label = e
                
                gen_sample_num = gen_sample_num + 1
            
                outtext.append(str(dpt)+"\t"+str(label)+"\t"+str(doc1))
                               #+"\t"+str(doc2))

    rng.shuffle(outtext)
    
    file = open(output_directory, "w")
    file.write("\n".join(outtext))
    file.close()
    
    
    rng.shuffle(outtext)
    
    file = open(output_directory+"+info.txt", "w")
    file.write(str(gen_sample_num))
    file.close()
    

# sampling train
sampling_data(docs_arrays=docs_train, samplenum=1000, dup=1, output_directory="./data/train.txt")

# sampling test
sampling_data(docs_arrays=docs_test, samplenum=200, dup=1, output_directory="./data/test.txt")
